#### Label extraction

In [1]:
import numpy as np
import os
import random
import scipy.signal as signal
from  scipy.signal import ShortTimeFFT
import scipy.signal.windows as wins
import matplotlib.pyplot as plt

In [2]:
# Parameters
t_sweep=0.001
n_rangebins=128
bandwidth=4e8

In [3]:
labels=[]
spectrograms = []

cwd = os.getcwd()
# change to the folder where the dataset is stored
base_folder="G:\\My Drive\\tudelft\\Q4\\Object Classification with Radar\\project_dataset"

subfolders = [f.path for f in os.scandir(base_folder) if f.is_dir() ]
filenames=[]

for folder in subfolders:
    files = [f.path for f in os.scandir(folder) if f.is_file()]
    filenames.extend(files)

In [4]:
# STFT params
winlength=128
overlap=0.5
nfft=2048
win=wins.hann(winlength)
hop=int(winlength*(1-overlap))

In [5]:
#Pick 20 random filenames
test_filenames=random.sample(filenames,20)
print(test_filenames)

# Print
for i, filename in enumerate(test_filenames):
    with open(filename, 'r') as file:
        data = np.loadtxt(file, delimiter=' ', skiprows=4, dtype=np.complex_, converters=lambda s: complex(s.decode().replace('i', 'j')))
        data_numsamples=len(data)
        data_time=data_numsamples*t_sweep/n_rangebins

        fs=int(n_rangebins/t_sweep)
        n_sweeps=data_time/t_sweep

        data = data.reshape(n_rangebins, int(n_sweeps), order='F')
        data_fft = np.fft.fftshift(np.fft.fft(data, axis=0), axes=0)
        data_fft = data_fft[int(n_rangebins/2):,:]
        b, a = signal.butter(4,0.0075, 'high')
        range_profile = signal.filtfilt(b, a, data_fft, axis=1)
        prof=np.sum(range_profile,axis=0)
        stft=ShortTimeFFT(win, hop, mfft=nfft, fs=fs, scale_to='magnitude', fft_mode='centered')
        nul,nul,f_lo,f_hi=stft.extent(int(fs*n_sweeps))
        Sxx = stft.spectrogram(prof,axis=0)
        spectrograms.append(Sxx)


    name_of_file = filename.split('\\')[-1][:-4]
    activity = int(name_of_file.split('P')[0])
    subject_id = int(name_of_file.split('P')[1].split('A')[0])
    activity_performed = int(name_of_file.split('P')[1].split('A')[1].split('R')[0])
    repetition = int(name_of_file.split('P')[1].split('A')[1].split('R')[1])

    labels.append([activity, subject_id, activity_performed, repetition])
    # print(name_of_file, activity, subject_id, activity_performed, repetition)
        

['G:\\My Drive\\tudelft\\Q4\\Object Classification with Radar\\project_dataset\\4 July 2018 Dataset\\6P62A06R2.dat', 'G:\\My Drive\\tudelft\\Q4\\Object Classification with Radar\\project_dataset\\1 December 2017 Dataset\\5P52A05R02.dat', 'G:\\My Drive\\tudelft\\Q4\\Object Classification with Radar\\project_dataset\\5 February 2019 UoG Dataset\\5P07A05R01.dat', 'G:\\My Drive\\tudelft\\Q4\\Object Classification with Radar\\project_dataset\\4 July 2018 Dataset\\3P65A03R1.dat', 'G:\\My Drive\\tudelft\\Q4\\Object Classification with Radar\\project_dataset\\5 February 2019 UoG Dataset\\4P10A04R01.dat', 'G:\\My Drive\\tudelft\\Q4\\Object Classification with Radar\\project_dataset\\7 March 2019 West Cumbria Dataset\\4P40A04R3.dat', 'G:\\My Drive\\tudelft\\Q4\\Object Classification with Radar\\project_dataset\\1 December 2017 Dataset\\6P43A06R01.dat', 'G:\\My Drive\\tudelft\\Q4\\Object Classification with Radar\\project_dataset\\7 March 2019 West Cumbria Dataset\\5P47A05R1.dat', 'G:\\My Drive\\

In [1]:
freq_axis=np.linspace(0,n_rangebins,n_rangebins)*fs/(2*n_rangebins)
range_axis=freq_axis*3e8*t_sweep/(2*bandwidth)
time_axis=np.arange(0, data_time, t_sweep)

activity_dict = {1: 'Walking', 2: 'Sitting Down', 3: 'Standing Up', 4: 'Picking up an Object', 5: 'Drinking Water', 6: 'Falling'}

for i in range(10):
    plt.figure()
    plt.imshow(20*np.log10(spectrograms[i]), aspect='auto', extent=[time_axis[0], time_axis[-1], f_lo, f_hi],origin='upper',interpolation='none',cmap='jet')
    plt.colorbar()
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)')
    plt.title(f"{activity_dict[labels[i][2]]}, Subject {labels[i][1]}, Repetition {labels[i][3]}")
    plt.show()

NameError: name 'np' is not defined